In [2]:
import numpy as np
hic = np.loadtxt('/data/ghy_data/GM12878/chr19_50000.hic')#Hi-c data
print(hic.shape)

(1182, 1182)


In [4]:
import numpy as np

hic = np.loadtxt('/data/ghy_data/GM12878/chr19_50000.hic')#Hi-c data
subcomparts = np.loadtxt('output/GM12878/50kb_TAD/chr19_subcomparts.txt')
tad = np.loadtxt('output/GM12878/50kb_TAD/chr19.txt')#此处用四列的TAD数据

outpath = "output/GM12878/hypergraph/hyperedge.txt"
i = 0
j = 0
nodes = []

with open(outpath, "w") as out:
    while i<len(subcomparts):
        start = int(subcomparts[i][0])
        end = int(subcomparts[i][2])
        while j<len(tad):
            if not(tad[j][2] + 3< start or tad[j][0] - end >3) :
                nodes.append(j+1)
            j = j+1
        if len(nodes)>0:
            out.write("\t".join((str(nodes[0]),  str(nodes[-1]))) + "\n")
        nodes = []
        j = 0
        i = i+1
    out.close()

In [5]:
import numpy as np

hyperedge = np.loadtxt("output/GM12878/hypergraph/hyperedge.txt")
tad = np.loadtxt('output/GM12878/50kb_TAD/chr19.txt')#此处用四列的TAD数据
outpath = "output/GM12878/hypergraph/hyperedge_all.txt"
i = 0
hyperedge2 = np.array([[0,0]])
if hyperedge[0][0]>1:
    hyperedge2 = np.array([[1,int(hyperedge[0][0])-1]])

while i < len(hyperedge)-1 :
    if int(hyperedge[i][1])+1 < int(hyperedge[i+1][0]):
        hyperedge2 = np.append(hyperedge2,np.array([[int(hyperedge[i][1])+1,int(hyperedge[i+1][0])-1]]),axis=0)
    i= i+1
hyperedge2 = np.delete(hyperedge2, 0,0)
hyperedge = np.append(hyperedge,hyperedge2,axis=0)
hyperedge = np.sort(hyperedge,axis=0)
np.savetxt(outpath, np.c_[hyperedge],fmt='%d',delimiter='\t')

In [7]:

import numpy as np

hyperedge = np.loadtxt("output/GM12878/hypergraph/hyperedge_all.txt")
outpath = "output/GM12878/hypergraph/hyperedge_flat.txt"

res = []
temp = []
i = 0

while i<len(hyperedge):
    temp = list(range(int(hyperedge[i][0])-1,int(hyperedge[i][1])))
    for _ in temp:
        res.append([_, i])
    temp = []
    i = i+1

res = np.array(res)
print(res.shape)

np.savetxt(outpath, res, fmt='%d',delimiter='\t')


(120, 2)


In [9]:
# 处理A/B区室结果
import numpy as np

# AB_KR = np.loadtxt('/home/zsc/study/biye/fanc/compartments/IMR90/chr19.txt')
A_KR_outpath = 'output/IMR90/AB/A_KR.txt'
B_KR_outpath = 'output/IMR90/AB/B_KR.txt'
with open('fanc/compartments/GM12878/chr19.txt') as nkFile:
    lines = [line.strip().split() for line in nkFile]
group = np.array(lines)
AB_KR = group[:,4]
# print(int(AB_KR[1]))
with open(A_KR_outpath, "w") as out:
    left = -1
    for i in range(len(AB_KR)):
        if AB_KR[i]>='0' and left==-1:
            left = i
        elif left>=0 and (AB_KR[i]<'0' or str(AB_KR[i])=="nan") :
            out.write("\t".join((str(left+1),str(i))) + "\n")
            left=-1
    if left>=0:
        out.write("\t".join((str(left+1),str(len(AB_KR)))) + "\n")

    # out.close()

with open(B_KR_outpath, "w") as out:
    left = -1
    for i in range(len(AB_KR)):
        if AB_KR[i]<'0' and left==-1:
            left = i
        elif left>=0 and (AB_KR[i]>'0' or str(AB_KR[i])=="nan") :
            out.write("\t".join((str(left+1),str(i))) + "\n")
            left=-1
    if left>=0:
        out.write("\t".join((str(left+1),str(len(AB_KR)))) + "\n")

    # out.close()

In [8]:
#先获取子区室结果
# 处理子区室结果

import numpy as np

A1_outpath = 'output/IMR90/sub-compartment/A1.txt'
B1_outpath = 'output/IMR90/sub-compartment/B1.txt'
A2_outpath = 'output/IMR90/sub-compartment/A2.txt'
B2_outpath = 'output/IMR90/sub-compartment/B2.txt'
with open('output/IMR90/sub-compartment/chr19_sub_compartments.bed') as nkFile:
    lines = [line.strip().split() for line in nkFile]
group = np.array(lines)
start = group[:,1]
end = group[:,2]
AB_KR = group[:,3]
with open(A2_outpath, "w") as out:
    left = -1
    for i in range(len(AB_KR)):
        if AB_KR[i][:3] == 'A.2' and left==-1:
            left = i
        elif left>=0 and (AB_KR[i][:3] != 'A.2' or str(AB_KR[i])=="nan") :
            out.write("\t".join((str(int((int(start[left])-1)/50000)+1),str(int(int(end[i-1])/50000)))) + "\n")
            left=-1
    # if left>=0:
    #     out.write("\t".join((str(int((int(start[left])-1)/50000)),str(1172))) + "\n")

In [14]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def normalization(data):
    _range = np.max(data)-np.min(data)
    return (data-np.min(data)) / _range

data_CTCF = np.load(
    'epi/output/IMR90/chr19_50000b_CTCF.npy')
data_H3K4me3 = np.load(
    'epi/output/IMR90/chr19_50000b_H3K4me3.npy')
data_H3K27ac = np.load(
    'epi/output/IMR90/chr19_50000b_H3K27ac.npy')
data_POLR2A = np.load(
    'epi/output/IMR90/chr19_50000b_POLR2A.npy')
data_H3K9me3 = np.load(
    'epi/output/IMR90/chr19_50000b_H3K9me3.npy')
print(data_H3K4me3.shape)
feature = np.dstack((data_H3K27ac[:1173], data_H3K4me3[:1173], data_CTCF[:1173], data_POLR2A[:1173],data_H3K9me3[:1173]))
print(feature[0])
# 特征归一化
# feature_norm = []
# for j in feature[0]:
#     feature_norm.append(normalization(j))
# np.savetxt("./output/feature_norm/chr"+str(i)+'.txt',
#            feature_norm, fmt="%f", delimiter=' ')
np.savetxt("epi/output/IMR90/chr19.txt",feature[0], fmt="%f", delimiter=' ')

(1172,)
[[ 0.07408     0.16294     0.10196     0.08117     0.06311   ]
 [ 0.08814944  0.16272644  0.11364739  0.08288952  0.08109683]
 [ 0.07420886  0.16294     0.11546627  0.0845498   0.06698602]
 ...
 [ 1.86968053  6.27048508  0.41338051  1.31347769  0.07955498]
 [ 3.4683096  10.11486858  3.07137689  2.63908803  0.27368701]
 [ 7.8469395  12.31846014  2.28528711  4.38680025  0.02484146]]


In [29]:
# ##19号染色体为例 ##计算每个TAD的特征值 ##TAD内节点特征值的均值
def feature_edge(feature_path,out_path):
    dataAll = []
    single_edge_feature = []
    feature_19 = np.loadtxt(feature_path)
    data_19 = np.loadtxt('output/IMR90/50kb_TAD/chr19.txt')

    f=open('output/IMR90/50kb_TAD/chr19') #此处用二列的TAD数据
    for line in f.readlines():
        dataSet = []
        for i in np.arange(int(line.split()[0]), int(line.split()[1])+1):
            dataSet.append(feature_19[i-1])
        single_TAD_feature = np.mean(dataSet, axis=0)
        dataAll.append(single_TAD_feature)
    f.close

    feature_norm = []
    for j in dataAll:
        feature_norm.append(normalization(j))

    np.savetxt(out_path, feature_norm, fmt="%f")
    # np.savetxt("./output/feature_edge/chr19/"+'5861_PCA.txt',
    #            feature_PCA, fmt="%f", delimiter=' ')

feature_edge('epi/output/IMR90/chr19.txt','epi/output/IMR90/chr19_TAD.txt')

In [22]:
# ##19号染色体为例 ##计算每个类TAD信号值和
def feature_edge(feature_path,out_path):
    dataAll = []
    single_edge_feature = []
    feature_19 = np.loadtxt(feature_path)
    data_19 = np.loadtxt('output/IMR90/50kb_TAD/chr19.txt')

    f=open('output/IMR90/hypergraph/two/second.txt')
    for line in f.readlines():
        start = int(line.split()[0])
        end = int(line.split()[1])
        # if start>=48 and end<=196:
        dataSet = []
        for i in np.arange(start, end+1):
            dataSet.append(feature_19[i-1])
        single_TAD_feature = np.sum(dataSet, axis=0)
        dataAll.append(single_TAD_feature)
    f.close
    v_H3K27ac = np.sum(dataAll, axis=0)[0]
    v_H3K4me3 = np.sum(dataAll, axis=0)[1]
    v_CTCF = np.sum(dataAll, axis=0)[2]
    v_POLR2A = np.sum(dataAll, axis=0)[3]
    v_H3K9me3 = np.sum(dataAll, axis=0)[4]
    return v_H3K27ac,v_H3K4me3,v_CTCF,v_POLR2A,v_H3K9me3

v_H3K27ac,v_H3K4me3,v_CTCF,v_POLR2A,v_H3K9me3 = feature_edge('epi/output/IMR90/chr19.txt','output/IMR90/hypergraph/two/chr19_TAD.txt')
print(v_H3K27ac,v_H3K4me3,v_CTCF,v_POLR2A,v_H3K9me3)

712.2800679999998 1414.1970250000004 540.1042300000001 577.3991860000001 490.795189
